<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/JNCLectures_Intro_to_ML/blob/main/Week9/Lec9_bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install xclip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xclip is already the newest version (0.13-2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install drawdata

In [ ]:
from drawdata import ScatterWidget
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score


In [ ]:
widget = ScatterWidget()
widget

ScatterWidget()

In [ ]:
data = widget.data_as_pandas
data.head()

,x,y,color,label
0,92.013880,56.302816,#1f77b4,a
1,76.704344,88.077088,#1f77b4,a
2,65.112312,100.750276,#1f77b4,a
3,21.392211,70.509797,#1f77b4,a
4,83.753352,88.505035,#1f77b4,a


In [ ]:
data["target"] = 0.1*(data["x"])/(data["x"]) + 0.5*data["y"]*np.log(data["x"]) + 0.4*data["x"]**2
data.head()

,x,y,color,label,target
0,92.013880,56.302816,#1f77b4,a,3514.020577
1,76.704344,88.077088,#1f77b4,a,2544.647971
2,65.112312,100.750276,#1f77b4,a,1906.317566
3,21.392211,70.509797,#1f77b4,a,291.137373
4,83.753352,88.505035,#1f77b4,a,3001.894233


In [ ]:
X = data[["x","y"]]
y = data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#### Baseline regressor

In [ ]:
# fit basic regression tree with depth 3
base_tree = DecisionTreeRegressor(max_depth=2, random_state=0)
base_tree.fit(X_train, y_train)
y_pred = base_tree.predict(X_test)

# metrics
r2_score(y_test, y_pred)

0.9579580870855693

In [ ]:
# create bootstrapped samples
# create a random number generator that will be used to set the randomness
rng = np.random.RandomState(42)

def get_bootstrap_data(X_train, y_train):

    train_indices = X_train.index
    bootstrap_indices = rng.choice(train_indices,
                                  len(train_indices),
                                  replace = True)
    ratio = len(set(train_indices).intersection(set(bootstrap_indices)))/len(train_indices)
    X_train_bootstrap = X_train.loc[bootstrap_indices]
    y_train_bootstrap = y_train.loc[bootstrap_indices]

    return X_train_bootstrap, y_train_bootstrap, ratio

In [ ]:
# Now, we take four trees and train on separate bootrapped samples.
# Final prediction is the average of all the 4 predictions for each test observation
n_tree = 4
y_pred = []
tree = DecisionTreeRegressor(max_depth=2, random_state=0)

for i in range(n_tree):
  X_train_bootstrap, y_train_bootstrap, _ = get_bootstrap_data(X_train = X_train, y_train = y_train)
  tree.fit(X_train_bootstrap, y_train_bootstrap)
  y_pred_bootstrap = tree.predict(X_test)
  y_pred.append(y_pred_bootstrap)

In [ ]:
# final prediction is average of bootstrapped tree predicitons
y_pred_final = np.mean(np.stack(np.array(y_pred).T, axis = 0), axis = 1)

# metrics
r2_score(y_test, y_pred_final)

0.9651903514869867

We have increased accuracy from 95.79% to 96.52% by bootstrapped aggregation.